##### Emanuele Anarratone 20050533
# Quantum Identity Authentication
## Simulazione del protocollo con intercettazione al mittente
### 1.Introduzione

Questo report analizza il comportamento del protocollo di Quantum Identity Authentication (QIA) in presenza di un attacco da parte di un intercettatore, identificato come Eve, posizionato sul canale del mittente.
L’attacco consiste nell’intercettazione e misurazione dello stato quantistico inviato da Alice, prima che questo raggiunga Bob.  
Successivamente, Eve tenta di riprodurre uno stato quantistico simile (ma non identico, a causa del principio di **no-cloning**) e lo invia a Bob, cercando così di impersonare Alice e accedere al canale di comunicazione autenticato.
Lo scopo di questa simulazione è verificare se il protocollo QIA è in grado di rilevare l'intercettazione tramite il degrado della correlazione tra i risultati di Alice e Bob

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session
from qiskit_aer import AerSimulator

import random, hmac, hashlib

In [ ]:
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="TOKEN",
    overwrite=True
)

service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")   #Inizializzazione del backend sulla macchina di Brisbane
aer = AerSimulator.from_backend(backend)    #Inizializzazione del simulatore con backend di una macchina reale per simulazioni veloci con simulazione del rumore


C:\Users\Emanuele\AppData\Local\Temp\ipykernel_8716\2930734499.py:7: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService()


In [5]:
# Funzione per HMAC classico
def compute_hmac(key: bytes, message: str) -> str:
    return hmac.new(key, message.encode(), hashlib.sha256).hexdigest()


In [6]:
rounds = 1000    
threshold = 0.95

alice_basis, bob_basis = [], []
alice_results, bob_results = [], []
eve_basis, eve_results = [], []
circuits = []

### 2. Intercettazione

Eve intercetta lo stato quantistico inviato da Alice ed effettua una misura.
Per semplicità, la misurazione di Eve è stata simulata utilizzando il simulatore **Aer** di Qiskit, con l’aggiunta di rumore modellato a partire dal backend della macchina quantistica reale di Brisbane.
A causa del principio di **collasso della funzione d’onda**, la misurazione di Eve proietta lo stato in uno dei possibili quattro risultati, distruggendo l’informazione quantistica originaria.

Per continuare la comunicazione con Bob, Eve deve preparare un nuovo stato quantistico da inviare, cercando di replicare quello che avrebbe dovuto ricevere da Alice. Tuttavia, a causa della natura degli **stati entangled**, i risultati delle misure tra le parti non saranno più fortemente correlati come in una comunicazione non intercettata, portando all’**invalidazione dell’autenticazione** da parte del server.

> **Nota**: a fini dimostrativi, ogni ciclo della simulazione rappresenta sia le operazioni svolte da Eve che la successiva comunicazione tra Alice e Bob.  
In particolare:
> - Il circuito `qc` rappresenta lo stato originale, **interrotto** dalla misura di Eve, ed è eseguito tramite simulazione con Aer.  
> - Il circuito `qc2` rappresenta il nuovo stato preparato da Eve e inviato a Bob, ed è **eseguito sulla macchina quantistica reale** tramite il `Sampler`.

In [7]:
for i in range(rounds):
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)

    alice_b = random.choice(['Z', 'X'])
    eve_b = random.choice(['Z', 'X'])
    bob_b = random.choice(['Z', 'X'])

    if alice_b == 'X':      
        qc.h(0)


    if eve_b == 'X':
        qc.h(1)
    
    qc.measure(0, 0)
    qc.measure(1, 1)

    tqc = transpile(qc, backend=backend)
    res = aer.run(tqc, shots=1).result().get_counts()
    key = list(res.keys())[0]
    alice_bit = int(key[-1])  # bit di Alice (registro classico 0)
    eve_bit = int(key[0])     # bit di Eve   (registro classico 1)


    # Eve prepara secondo la sua base e risultato
    qc2 = QuantumCircuit(2, 2)
    if eve_b == 'Z':
        if eve_bit == 1:
            qc2.x(0)
        # else: lascia |0>
    else:  # X
        qc2.h(0)
        if eve_bit == 1:
            qc2.x(0)
        qc2.h(0)

    # Bob misura il qubit nella sua base

    if bob_b == 'X':
        qc2.h(1)

    qc2.measure(1, 1)
    circuits.append(qc2)    #La vera misura per il canale viene fatta sulla comunicazione a Bob
    alice_basis.append(alice_b)
    bob_basis.append(bob_b)
    eve_basis.append(eve_b)

In [8]:
with Session(backend=backend) as session:
    sampler = Sampler(mode=backend)
    t_circuits = transpile(circuits, backend=backend)
    job = sampler.run(t_circuits, shots=1)
    results = job.result()

    for i, bitarray in enumerate(results._pub_results):
        bitstring = bitarray.data.c.get_bitstrings()[0]
        alice_results.append(bitstring[0])
        bob_results.append(bitstring[1])
        # Eve "misura" lo stesso bit che Alice
        eve_results.append(bitstring[0])

base_primitive._get_mode_service_backend:WARNING:2025-06-11 23:32:28,291: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.
c:\Users\Emanuele\miniconda3\envs\qc\Lib\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:1129: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)


### 3. Autenticazione
Nel presente esempio viene simulato lo scenario in cui Eve tenta di autenticarsi presso il server utilizzando il messaggio intercettato da Alice per generare un HMAC valido.
Tuttavia, nel protocollo QIA, il superamento della verifica delle firme (HMAC) **non è sufficiente** per ottenere l’accesso al canale autenticato: è necessaria anche una **forte correlazione** tra i risultati delle misurazioni di Alice e Bob, entro i limiti imposti dal valore di soglia `threshold`.
Poiché Eve ha alterato lo stato quantistico originale con la propria misura e successiva ricostruzione, la correlazione tra le misure risulterà degradata, causando il **fallimento dell'autenticazione** nonostante l’HMAC possa essere formalmente corretto.

In [9]:
K_A = b'chiavealice'
K_B = b'chiavedibob'
K_E = b'chiavealice'   #Eve usa la chiave di Alice per autenticarsi


msg_A = ''.join(alice_basis) + ''.join(alice_results)
msg_B = ''.join(bob_basis) + ''.join(bob_results)
msg_E = ''.join(alice_basis) + ''.join(alice_results)  # Eve usa i dati di Alice per autenticarsi

hmac_A = compute_hmac(K_A, msg_A)
hmac_B = compute_hmac(K_B, msg_B)
hmac_E = compute_hmac(K_E, msg_E)

### 4. Server
Con un numero elevato di `rounds`, necessario per ottenere una maggiore precisione statistica, ci si aspetta che la **correlazione tra i risultati** successiva a un'intercettazione da parte di Eve sia mediamente intorno al **75%**.
Di conseguenza, Eve **non riuscirà ad autenticarsi** al canale nei panni di Alice, in quanto la soglia `threshold` richiesta per l’autenticazione (in questo caso ≥ 95%) non verrà raggiunta.
Inoltre, poiché la misurazione eseguita da Eve **collassa lo stato quantistico originale**, anche la correlazione tra Alice e Bob risulterà degradata e si attesterà su valori simili (≈75%).  
Questo permette al server di **rilevare l’intercettazione** e invalidare la comunicazione, garantendo così la sicurezza del canale quantistico.

In [11]:
valid_A = (compute_hmac(K_A, msg_A) == hmac_A)
valid_B = (compute_hmac(K_B, msg_B) == hmac_B)
valid_E = (compute_hmac(K_E, msg_E) == hmac_A) #Eve vuole autenticarsi come alice

used_ab, matches_ab = 0, 0
used_eb, matches_eb = 0, 0

for a_b, b_b, a_r, b_r, e_b, e_r in zip(alice_basis, bob_basis, alice_results, bob_results, eve_basis, eve_results):
    # Alice-Bob
    if a_b == b_b:
        used_ab += 1
        if a_r == b_r:
            matches_ab += 1
    # Eve-Bob
    if e_b == b_b:
        used_eb += 1
        if e_r == b_r:
            matches_eb += 1

match_ratio_ab = matches_ab / used_ab if used_ab else 0
match_ratio_eb = matches_eb / used_eb if used_eb else 0
authenticated = valid_A and valid_B and match_ratio_ab >= threshold
authenticated_eb = valid_E and valid_B and match_ratio_eb >= threshold


output = f"""
HMAC Alice: {hmac_A}
HMAC Bob:   {hmac_B}
HMAC Eve:   {hmac_E}
HMAC Alice valido: {valid_A}
HMAC Bob valido:   {valid_B}
HMAC Eve valido:   {valid_E}

Bits usati (Alice-Bob): {used_ab}
Coincidenze Alice-Bob: {matches_ab}
Match % Alice-Bob: {match_ratio_ab*100:.2f}%

Bits usati (Eve-Bob): {used_eb}
Coincidenze Eve-Bob: {matches_eb}
Match % Eve-Bob: {match_ratio_eb*100:.2f}%

Autenticazione Alice-bob {"riuscita" if authenticated else "non riuscita"}
Autenticazione Eve-Bob {"riuscita" if authenticated_eb else "non riuscita"}
"""

print(output)


HMAC Alice: 65eb32ae9b9179ba10acdaa0909292748651c9facb51a4c3e1f3f65c011e24c7
HMAC Bob:   f4bc16a4856ca840e1e7af2d9c13e28d130530949c8001918b12554a6e42beb5
HMAC Eve:   65eb32ae9b9179ba10acdaa0909292748651c9facb51a4c3e1f3f65c011e24c7
HMAC Alice valido: True
HMAC Bob valido:   True
HMAC Eve valido:   True

Bits usati (Alice-Bob): 469
Coincidenze Alice-Bob: 336
Match % Alice-Bob: 71.64%

Bits usati (Eve-Bob): 493
Coincidenze Eve-Bob: 365
Match % Eve-Bob: 74.04%

Autenticazione Alice-bob non riuscita
Autenticazione Eve-Bob non riuscita



### 5. Analisi dell'output - Conclusioni

Nel caso in cui Eve intercetti il messaggio inviato da Alice, essa ottiene anche le basi di misura e i relativi risultati, diventando così in grado di calcolare un HMAC valido. Il server, ricevendo l’HMAC generato da Eve, lo considera coerente con quello atteso per Alice e autorizza la comunicazione, ritenendola legittima. Questo rappresenta lo scenario peggiore per la sicurezza del protocollo, poiché l’intercettazione avviene senza che Bob o il server possano rilevarla. È proprio per prevenire questo tipo di attacchi che si ricorre all’uso delle coppie di stati entangled, le quali rendono le misurazioni non replicabili da un attaccante esterno.
Nel contesto sperimentale considerato, è stata analizzata una comunicazione originale tra Alice e Bob e una seconda comunicazione intercettata tra Eve e Bob. Nella prima, vengono utilizzati 469 bit per il confronto, dei quali 336 risultano coincidenti, con un tasso di corrispondenza del 71,64%, insufficiente per superare la soglia di autenticazione. Poiché Alice e Bob si fidano reciprocamente, un risultato inferiore alla soglia rappresenta un chiaro indicatore dell’intercettazione della comunicazione.
Nella comunicazione intercettata, invece, vengono utilizzati 493 bit, con 365 coincidenze. Poiché Eve non è parte del processo entangled, ogni tentativo di misurazione corrisponde a un’estrazione casuale, con una probabilità del 50% di ottenere un risultato errato. Di conseguenza, il tasso di corrispondenza si mantiene attorno al 75%, confermando l’aspettativa teorica. Questo comportamento evidenzia come, in presenza di un attacco man-in-the-middle, l’uso di coppie entangled garantisca un riconoscimento affidabile tra le parti e permetta di identificare comunicazioni compromesse anche su un canale insicuro.